##### Copyright 2022 The Cirq Developers

In [5]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# braiding protocol on the 4-edge 'glasses' setup

Prepares the ground state for the 'glasses' lattice, applies braiding protocol and measures. Qubit grid layout is specified [here](https://drive.google.com/file/d/1FtM2V43WLXdzs4QjoLTaEO469e4BRhbZ/view?usp=sharing).

In [6]:
# Set the measurement you want to either 'charge' or 'flux' or 'both'.
measurement_type = 'both'

# Set the initial state to 'identity', 'ground state' or 'random'

initial_state = 'ground state'

# Set whether or not a ribbon is applied (True/False)

ribbon = True

# There are two possible ribbons: '1' corresponds to the interacting braids, '2' to the two unlinked loops
ribbon_order = '1'

# Number of repetitions

repetitions = 100



## **Install** Cirq and qsim

In [7]:
# @title Install `cirq_google` and `qsimcirq`

try:
    import cirq
    import cirq_google
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq-google
    print("installed cirq.")
    import cirq
    import cirq_google

try:
    import qsimcirq
except ImportError:
    print("installing qsimcirq...")
    !pip install --quiet qsimcirq
    print(f"installed qsimcirq.")
    import qsimcirq

# Other modules used in this colab
import matplotlib.pyplot as plt
import time

## Create a **Quantum Virtual Machine**

The following cell builds a Quantum Virtual Machine that mimics a particular Google quantum hardware device (currently Rainbow or Weber) using the following customizable steps: 
- Constructing a `cirq.NoiseModel` object from device calibration data saved in Cirq. See [Representing Noise](/cirq/noise/representing_noise) for more on noise models. 
- Building a `qsimcirq.QsimSimulator` that uses this noise model. See [Noisy Simulation](/cirq/simulate/noisy_simulation) and [Noise simulation with qsim](/qsim/tutorials/noisy_qsimcirq) for more. 
- Creating a `cirq.Device` that imposes the same constraints on circuits that the original device would. See [Devices](/cirq/hardware/devices) for more on these constraint objects. 
- Packaging the simulator and device into an object that implements the `cirq.Engine` interface that the hardware device would use. 

If you don't need this level of control, you can also instantiate a QVM with `cirq_google.engine.create_default_noisy_quantum_virtual_machine`, as in [QVM Creation Template](/cirq/simulate/qvm_builder_code). 

In [8]:
# @title Choose a processor ("rainbow" or "weber")
processor_id = "weber"  # @param {type:"string"}

# Construct a simulator with a noise model based on the specified processor.
cal = cirq_google.engine.load_median_device_calibration(processor_id)
noise_props = cirq_google.noise_properties_from_calibration(cal)
noise_model = cirq_google.NoiseModelFromGoogleNoiseProperties(noise_props)
sim = qsimcirq.QSimSimulator(noise=noise_model)

# Create a device from the public device description
device = cirq_google.engine.create_device_from_processor_id(processor_id)
# Build the simulated local processor from the simulator and device.
sim_processor = cirq_google.engine.SimulatedLocalProcessor(
    processor_id=processor_id, sampler=sim, device=device, calibrations={cal.timestamp // 1000: cal}
)
# Package the processor to use an Engine interface
sim_engine = cirq_google.engine.SimulatedLocalEngine([sim_processor])
print(
    "Your quantum virtual machine",
    processor_id,
    "is ready, here is the qubit grid:",
    "\n========================\n",
)
print(sim_engine.get_processor(processor_id).get_device())

AttributeError: module 'cirq_google.engine' has no attribute 'load_median_device_calibration'

## **Create** a circuit, **transform** it (to make it executable on Google quantum hardware) and **choose qubits** on the processor. 

The circuit you use needs to be _device ready_, which means it: 
- Is comprised of operations from the device's gate set. 
- Is applied to qubits that exist on the device. 
- Respects the connectivity of qubits on the device.

Below is an example of a circuit that has the correct topology to be placed on the Weber device, and how it is prepared to be run on the QVM.

# Group structure

The multiplication matrices for $D_4$, to easily make the flux measurements at the end.


In [ ]:
# These permutation matrices compute left multiplication and right multiplication on the group elements as column vectors

import numpy as np


e = np.identity(8)

R_l = np.array([[0,0,0,1,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,1,0,0,0]])

R2  = np.array([[0,0,1,0,0,0,0,0],
                [0,0,0,1,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,1,0,0,0],
                [0,0,0,0,0,1,0,0]])

R3_l = np.array([[0,1,0,0,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,0,0,1,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,1,0,0,0],
                [0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0]])

m_l = np.array([[0,0,0,0,1,0,0,0],
                [0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,1],
                [1,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,0,0,1,0,0,0,0]])

mR_l = np.array([[0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,1,0,0,0],
                [0,0,0,1,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,1,0,0,0,0,0]])

mR2_l = np.array([[0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,0,0,1],
                  [0,0,0,0,1,0,0,0],
                  [0,0,0,0,0,1,0,0],
                  [0,0,1,0,0,0,0,0],
                  [0,0,0,1,0,0,0,0],
                  [1,0,0,0,0,0,0,0],
                  [0,1,0,0,0,0,0,0]])

mR3_l = np.array([[0,0,0,0,0,0,0,1],
                  [0,0,0,0,1,0,0,0],
                  [0,0,0,0,0,1,0,0],
                  [0,0,0,0,0,0,1,0],
                  [0,1,0,0,0,0,0,0],
                  [0,0,1,0,0,0,0,0],
                  [0,0,0,1,0,0,0,0],
                  [1,0,0,0,0,0,0,0]])

R_r = np.array([[0,0,0,1,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,1,0,0,0],
                [0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0]])

R3_r = np.array([[0,1,0,0,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,0,0,1,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,1,0,0,0]])

m_r = np.array([[0,0,0,0,1,0,0,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,1,0,0],
                [1,0,0,0,0,0,0,0],
                [0,0,0,1,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,1,0,0,0,0,0,0]])

mR_r = np.array([[0,0,0,0,0,1,0,0],
                [0,0,0,0,1,0,0,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,0,0,1,0],
                [0,1,0,0,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,0,0,1,0,0,0,0],
                [0,0,1,0,0,0,0,0]])

mR2_r = np.array([[0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,1,0,0],
                  [0,0,0,0,1,0,0,0],
                  [0,0,0,0,0,0,0,1],
                  [0,0,1,0,0,0,0,0],
                  [0,1,0,0,0,0,0,0],
                  [1,0,0,0,0,0,0,0],
                  [0,0,0,1,0,0,0,0]])

mR3_r = np.array([[0,0,0,0,0,0,0,1],
                  [0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,1,0,0],
                  [0,0,0,0,1,0,0,0],
                  [0,0,0,1,0,0,0,0],
                  [0,0,1,0,0,0,0,0],
                  [0,1,0,0,0,0,0,0],
                  [1,0,0,0,0,0,0,0]])

inv = np.array([[1,0,0,0,0,0,0,0],
                [0,0,0,1,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,0,0,1,0,0,0],
                [0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,1]])

# This function transforms group elements from string format (as written at start of file) to column vector format used in the code. 
# This makes some later functions more readable.

def vector(element):
    if element in ['e']:
        return np.array([1,0,0,0,0,0,0,0])
    elif element in ['R']:
        return np.array([0,1,0,0,0,0,0,0])
    elif element in ['R2']:
        return np.array([0,0,1,0,0,0,0,0])
    elif element in ['R3']:
        return np.array([0,0,0,1,0,0,0,0])
    elif element in ['m']:
        return np.array([0,0,0,0,1,0,0,0])
    elif element in ['mR']:
        return np.array([0,0,0,0,0,1,0,0])
    elif element in ['mR2']:
        return np.array([0,0,0,0,0,0,1,0])
    elif element in ['mR3']:
        return np.array([0,0,0,0,0,0,0,1])

def conjugacy_class(element):
    if element in ['e']:
        return [vector('e')]
    elif element in ['R2']:
        return [vector('R2')]
    elif element in ['R','R3']:
        return [vector('R'),vector('R3')]
    elif element in ['m','mR2']:
        return [vector('m'),vector('mR2')]
    elif element in ['mR','mR3']:
        return [vector('mR'),vector('mR3')]

  # Gets the correct matrices for left and right multiplication

def left_mult(element):
    if np.array_equal(element,np.array([1,0,0,0,0,0,0,0])) == True:
        return e
    elif np.array_equal(element,np.array([0,1,0,0,0,0,0,0])) == True:
        return R_l
    elif np.array_equal(element,np.array([0,0,1,0,0,0,0,0])) == True:
        return R2
    elif np.array_equal(element,np.array([0,0,0,1,0,0,0,0])) == True:
        return R3_l
    elif np.array_equal(element,np.array([0,0,0,0,1,0,0,0])) == True:
        return m_l
    elif np.array_equal(element,np.array([0,0,0,0,0,1,0,0])) == True:
         return mR_l
    elif np.array_equal(element,np.array([0,0,0,0,0,0,1,0])) == True:
        return mR2_l
    elif np.array_equal(element,np.array([0,0,0,0,0,0,0,1])) == True:
        return mR3_l

def right_mult(element):
    if np.array_equal(element,np.array([1,0,0,0,0,0,0,0])) == True:
        return e
    elif np.array_equal(element,np.array([0,1,0,0,0,0,0,0])) == True:
        return R_r
    elif np.array_equal(element,np.array([0,0,1,0,0,0,0,0])) == True:
        return R2
    elif np.array_equal(element,np.array([0,0,0,1,0,0,0,0])) == True:
        return R3_r
    elif np.array_equal(element,np.array([0,0,0,0,1,0,0,0])) == True:
        return m_r
    elif np.array_equal(element,np.array([0,0,0,0,0,1,0,0])) == True:
        return mR_r
    elif np.array_equal(element,np.array([0,0,0,0,0,0,1,0])) == True:
        return mR2_r
    elif np.array_equal(element,np.array([0,0,0,0,0,0,0,1])) == True:
        return mR3_r


# Circuit

The circuit that prepares the ground state. See [pdf](https://drive.google.com/file/d/1FtM2V43WLXdzs4QjoLTaEO469e4BRhbZ/view?usp=sharing) for diagram and numbering. Some notes on qubit encoding:

- The first 12 qubits represent the four edges. The first 3 represent edge 1 as per the diagram, etc. Within each triplet, the first qubit encodes $m$, the second $R^2$ and the third $R$. This allows the group element encoded to be recovered for flux measurement with a simple kronecker product.

- Qubits 13-16 are used for the charge measurement onto the plaquette made up of edges 3 and 4. Two qubits are needed to create a superposition of a four-element subgroup, and the other two are used to move the ancillas around and use parallel operations to reduce circuit depth.

- Qubits 17-21 are used for the two ribbons. Qubits 17-18 and 19-20 represent the two initial Bell pairs. The final qubit are used for moving them around.

In [ ]:
from cirq import H, CNOT, X, SWAP

# The first 18 qubits represent the edges, the additional ones are all ancillas.
qubits = cirq.LineQubit.range(21)

bowtie = cirq.Circuit()

if initial_state == 'ground state':

  # Set up superposition on the first and fourth edge

  bowtie.append([H(qubits[0]),
                H(qubits[1]),
                H(qubits[2]),
                H(qubits[9]),
                H(qubits[10]),
                H(qubits[11])])

  # Multiply onto the second and third edge with CNOTs

  bowtie.append([CNOT(qubits[0],qubits[3]),
                CNOT(qubits[1],qubits[4]),
                CNOT(qubits[2],qubits[5]),
                CNOT(qubits[9],qubits[6]),
                CNOT(qubits[10],qubits[7]),
                CNOT(qubits[11],qubits[8])])

if initial_state == 'random':

  omegas = [np.random.rand() * np.pi * 2 for i in range(23)]
  phis = [np.random.rand() * np.pi * 2 for i in range(23)]
  thetas = [np.arccos(2 * np.random.rand() - 1) for i in range(23)]

  bowtie.append([cirq.rz(omegas[i])(qubits[i]) for i in range(23)])
  bowtie.append([cirq.rx(thetas[i])(qubits[i]) for i in range(23)])
  bowtie.append([cirq.rz(phis[i])(qubits[i]) for i in range(23)])


# Ribbon operators

Then the ribbon operators. They both use two ancillas in a Bell state, with one of the ancillas moved around the circuit and interacting with the edges. Multiplication circuits for the conjugacy class $\{R,R^3\}$ are given [here](https://drive.google.com/file/d/18bPluU4Pmz3h7Fi925FoPnR66rskn_Ef/view?usp=sharing).

In [ ]:
if ribbon and ribbon_order == '1':

  # Set up two bell pairs

  bowtie.append([H(qubits[16]),                                     # Blue ribbon
                CNOT(qubits[16],qubits[17])])

  bowtie.append([H(qubits[18]),                                     # Red ribbon
                CNOT(qubits[18],qubits[19])])

  # Create two particle pairs: multiply blue into edge 4 and red into edge 1

  bowtie.append([CNOT(qubits[17],qubits[10]),                       # blue onto edge 4
                SWAP(qubits[10],qubits[11]),
                CNOT(qubits[9],qubits[11]),

                CNOT(qubits[11],qubits[10]),                       # SWAP + CNOT = 2 CNOTs
                CNOT(qubits[10],qubits[11]),
                X(qubits[11]),
                        
                SWAP(qubits[19],qubits[0]),                         # red onto edge 1
                CNOT(qubits[2],qubits[1]),
                CNOT(qubits[0],qubits[1]),
                X(qubits[1])                
                ])                                                  # note: qubits 19 and 0 not yet swapped back

  # Sigma 1

  bowtie.append([SWAP(qubits[9],qubits[11]),
                SWAP(qubits[10],qubits[7]),
                CNOT(qubits[11],qubits[16]),                        # Conjugate back of ribbon by edge 4
                CNOT(qubits[17],qubits[10]),                        # Start multiplying front into edge 3
                SWAP(qubits[9],qubits[11]),
                SWAP(qubits[10],qubits[7]),

                SWAP(qubits[10],qubits[11]),                        # multiply front into edge 3, back into edge 4
                SWAP(qubits[7],qubits[8]),
                CNOT(qubits[16],qubits[11]),
                CNOT(qubits[6],qubits[8]),

                CNOT(qubits[11],qubits[10]),
                CNOT(qubits[10],qubits[11]),
                CNOT(qubits[8],qubits[7]),
                CNOT(qubits[7],qubits[8]),

                X(qubits[8]),
                X(qubits[10]),
                X(qubits[11]),

                SWAP(qubits[16],qubits[20]),                         # move back of ancilla and conjugate by edge 4
                CNOT(qubits[9],qubits[20])

                ])

  # Sigma 2

  bowtie.append([CNOT(qubits[3],qubits[0]),                           # Conjugate red by edge 2, swap back
                SWAP(qubits[19],qubits[0]),                
                CNOT(qubits[9],qubits[20]),                          # Conjugate blue by edge 4

                SWAP(qubits[19],qubits[17]),                         # Cross over: front ends of two ribbons go past each other
                SWAP(qubits[0],qubits[1]),
                CNOT(qubits[1],qubits[20]),                          # Conjugate blue by edge 1

                SWAP(qubits[0],qubits[1]),
                SWAP(qubits[17],qubits[16]),
                SWAP(qubits[6],qubits[8]),
                SWAP(qubits[8],qubits[11]),
                CNOT(qubits[11],qubits[16]),                         # Conjugate red by edge 3
                SWAP(qubits[8],qubits[11]),
                SWAP(qubits[6],qubits[8])
                ])

  # Fusion 1

  bowtie.append([SWAP(qubits[9],qubits[11]),
                CNOT(qubits[11],qubits[16]),                          # Conjugate red by edge 4
                CNOT(qubits[0],qubits[16]),                           # Conjugate by edge 1
                SWAP(qubits[9],qubits[11])
                ])

  # Fusion 2

  bowtie.append([SWAP(qubits[3],qubits[4]),                             # Multiply back of red into edge 2
                CNOT(qubits[18],qubits[3]),

                SWAP(qubits[3],qubits[4]),

                CNOT(qubits[5],qubits[4]),
                X(qubits[5]),
                CNOT(qubits[3],qubits[18]),                            # Conjugate red by edge 2
                
                SWAP(qubits[18],qubits[19]),
                CNOT(qubits[8],qubits[7]),
                SWAP(qubits[19],qubits[17]),
                SWAP(qubits[7],qubits[10]),
                CNOT(qubits[17],qubits[10]),                           # Multiply back of red into edge 3
                SWAP(qubits[7],qubits[10]),
                X(qubits[7]),
                X(qubits[8]),

                SWAP(qubits[6],qubits[8]),
                SWAP(qubits[17],qubits[16]),
                SWAP(qubits[8],qubits[11]),
                CNOT(qubits[11],qubits[16]),                           # Conjugate red by edge 3
                SWAP(qubits[8],qubits[11]),
                SWAP(qubits[6],qubits[8])
                ])

  # Measuring ribbons in bell state. Must bring ribbons back together first

  # Blue: back is at qubits 20, front is at qubit 18
  # Red: front is at qubit 17, back is at qubit 16

  bowtie.append([SWAP(qubits[18],qubits[19]),
                SWAP(qubits[19],qubits[17]),
                SWAP(qubits[17],qubits[16]),

                CNOT(qubits[20],qubits[16]),
                H(qubits[20]),

                CNOT(qubits[17],qubits[19]),
                H(qubits[17])
                ])


In [ ]:
if ribbon and ribbon_order == '2':

  # Set up two bell pairs

  bowtie.append([H(qubits[20]),                                     # Blue ribbon
                CNOT(qubits[20],qubits[16])])

  bowtie.append([H(qubits[18]),                                     # Red ribbon
                CNOT(qubits[18],qubits[19])])

  # Create two particle pairs: multiply blue into edge 4 and red into edge 1

  bowtie.append([SWAP(qubits[10],qubits[11]),
                 CNOT(qubits[16],qubits[11]),                      # blue onto edge 4
                CNOT(qubits[9],qubits[11]),

                CNOT(qubits[11],qubits[10]),                       # SWAP + CNOT = 2 CNOTs
                CNOT(qubits[10],qubits[11]),
                X(qubits[11]),
                         
                SWAP(qubits[19],qubits[0]),                        # red onto edge 1
                CNOT(qubits[2],qubits[1]),
                CNOT(qubits[0],qubits[1]),
                X(qubits[2])                                        # note: qubits 19 and 0 not yet swapped back
                ])

  # Sigma 2

  bowtie.append([CNOT(qubits[3],qubits[0]),                         # conjugate red by edge 2
                 SWAP(qubits[19],qubits[0]),
                 SWAP(qubits[9],qubits[11]),
                 SWAP(qubits[19],qubits[17]),
                 CNOT(qubits[11],qubits[16]),                       # conjugate blue by edge 4 and then 1
                 CNOT(qubits[0],qubits[16]),
                 SWAP(qubits[9],qubits[11]),
                 SWAP(qubits[6],qubits[8]),
                 SWAP(qubits[8],qubits[7]),
                 SWAP(qubits[17],qubits[10]),
                 CNOT(qubits[7],qubits[10])                         # conjugate red by edge 3

                ])
# have not swapped everything back yet!!


  # Sigma 1

  bowtie.append([CNOT(qubits[9],qubits[20]),                          # conjugate back of blue by edge 4
               
               CNOT(qubits[6],qubits[8]),                           # multiply red into edge 3
               CNOT(qubits[8],qubits[7]),                           # SWAP + CNOT = 2 CNOTs
               CNOT(qubits[7],qubits[8]),
               
               CNOT(qubits[10],qubits[7]),
               SWAP(qubits[6],qubits[8]),
               X(qubits[8]),

               SWAP(qubits[20],qubits[16]),                         # swap front and back of ribbon around             
               CNOT(qubits[16],qubits[17]),                         # multiply back of blue into edge 4, R2 qubit in position 17 still
               SWAP(qubits[17],qubits[10]),                         # move front of red ribbon back to 17
               CNOT(qubits[11],qubits[10]),
               X(qubits[11]),
               X(qubits[10]),
               
               SWAP(qubits[9],qubits[11]),
               CNOT(qubits[11],qubits[16])                          # and conjugate the ancilla by edge 4 again

               ])
  # qubits representing m and R on edge 4 are still swapped

  # Fusion 1

  bowtie.append([CNOT(qubits[11],qubits[16]),                       # conjugate ancilla by edge 4 again
                 SWAP(qubits[9],qubits[11]),                        # SWAP back
                 CNOT(qubits[0],qubits[16])                         # conjugate back of blue ribbon by edge 1
                 ])

  # Fusion 2

  bowtie.append([SWAP(qubits[3],qubits[4]),                             # Multiply back of red into edge 2
                CNOT(qubits[18],qubits[3]),

                SWAP(qubits[3],qubits[4]),

                CNOT(qubits[5],qubits[4]),
                X(qubits[5]),
                CNOT(qubits[3],qubits[18]),                            # Conjugate red by edge 2
                
                SWAP(qubits[18],qubits[19]),
                CNOT(qubits[8],qubits[7]),
                SWAP(qubits[19],qubits[17]),
                SWAP(qubits[17],qubits[10]),
                CNOT(qubits[10],qubits[7]),                           # Multiply back of red into edge 3
                X(qubits[7]),
                X(qubits[8]),
                 
                SWAP(qubits[6],qubits[9]),
                SWAP(qubits[9],qubits[11]),
                CNOT(qubits[11],qubits[10]),                          # conjugate red by edge 3 and move it all back
                SWAP(qubits[9],qubits[11]),
                SWAP(qubits[6],qubits[9]),
                SWAP(qubits[17],qubits[10])
                ])

  # Measuring ribbons in bell state.

  # Blue: back is at qubits 16, front is at qubit 21
  # Red: front is at qubit 19, back is at qubit 17

  bowtie.append([CNOT(qubits[16],qubits[20]),
                 H(qubits[16]),

                 CNOT(qubits[17],qubits[19]),
                 H(qubits[17])
                 ])


# Measurement

Then add measurements. Flux is measured by simply measuring qubits in the computational, charge is measured with a somewhat more complicated protocol. It uses the subgroup $\{e,m,R^2,mR^2\}$, which means it detects certain charges only. Additionally, the four qubits corresponding to the ribbon ancillas are measured, and post selection ensures the ribbons have projected out correctly.

In [ ]:
from cirq.circuits.insert_strategy import InsertStrategy
# Charge measurement setup for edges 3 and 4

if measurement_type == 'flux':
  if ribbon:
    bowtie.append(cirq.measure(*(qubits[0:12]+[qubits[20],qubits[16],qubits[17],qubits[19]]),key='out'))
  else:
    bowtie.append(cirq.measure(*qubits[0:12],key='out'))

if measurement_type == 'charge':
  bowtie.append([H(qubits[12]),H(qubits[14]),        # Set up superposition of subgroup {e,m,R2,mR2}. Qubit 12 encodes m, qubit 14 encodes R2.

                 CNOT(qubits[12],qubits[13]),         # Multiply onto neighbouring ancillas
                 CNOT(qubits[14],qubits[15]),

                 CNOT(qubits[12],qubits[9]),          # Multiply into the edges 3 and 4 that make up a plaquette, acting on the m and R2 qubits.
                 CNOT(qubits[13],qubits[6]),
                 CNOT(qubits[14],qubits[10]),
                 CNOT(qubits[15],qubits[7]),

                 CNOT(qubits[12],qubits[13]),         # Undo multiplication onto the additional ancillas
                 CNOT(qubits[14],qubits[15]),
  
                 H(qubits[12]),H(qubits[14])],strategy=InsertStrategy.NEW_THEN_INLINE)        # Hadamard back
  
  # measure the two qubits encoding the subgroup
  if ribbon:
    bowtie.append(cirq.measure(*([qubits[12],qubits[14]] + [qubits[20],qubits[16],qubits[17],qubits[19]]),key='out'))
  else:
    bowtie.append(cirq.measure(*([qubits[12],qubits[14]]),key='out'))

if measurement_type == 'both':
  bowtie.append([H(qubits[12]),H(qubits[14]),        # Set up superposition of subgroup {e,m,R2,mR2}. Qubit 12 encodes m, qubit 14 encodes R2.

                 CNOT(qubits[12],qubits[13]),         # Multiply onto neighbouring ancillas
                 CNOT(qubits[14],qubits[15]),

                 CNOT(qubits[12],qubits[9]),          # Multiply into the edges 3 and 4 that make up a plaquette, acting on the m and R2 qubits.
                 CNOT(qubits[13],qubits[6]),
                 CNOT(qubits[14],qubits[10]),
                 CNOT(qubits[15],qubits[7]),

                 CNOT(qubits[12],qubits[13]),         # Undo multiplication onto the additional ancillas
                 CNOT(qubits[14],qubits[15]),
  
                 H(qubits[12]),H(qubits[14])],strategy=InsertStrategy.NEW_THEN_INLINE)        # Hadamard back

  if ribbon:
    bowtie.append(cirq.measure(*(qubits[6:12] + [qubits[12],qubits[14]] + [qubits[20],qubits[16],qubits[17],qubits[19]]),key='out'))  
  else:
    bowtie.append(cirq.measure(*(qubits[6:12] + [qubits[12],qubits[14]]),key='out'))

print(bowtie)




           ┌───┐   ┌──┐       ┌──┐   ┌──┐   ┌──┐       ┌───┐                       ┌──┐   ┌──┐   ┌──┐                   ┌───┐       ┌──┐           ┌──┐               ┌────┐
0: ────H────@────────×────@─────X──────×─────×──────────────────────────────────────────────────────────×────────────────@────────────────────────────────────────────────────────────────────────
            │        │    │     │      │     │                                                          │                │
1: ────H────┼@──────X┼────X────X┼──────┼─────×────────────────────────────────────────────────────@─────×────────────────┼────────────────────────────────────────────────────────────────────────
            ││      ││          │      │                                                          │                      │
2: ────H────┼┼@─────@┼──────────┼──────┼──────────────────────────────────────────────────────────┼──────────────────────┼─────────────────────────────────────────────────────────────────────

### Transform the circuit 

In order to be executed on the Virtual Machine, the circuit must be transformed to a gateset supported by these machines. I have found three that can be used, which differ in the 2 qubit gate they use:

cirq.SqrtIswapTargetGateset uses the 'square root iSWAP' gate, a type of fSIM gate

cirq_google.SycamoreTargetGateset uses the 'Sycamore' gate, also fSIM type

cirq_google.GoogleCZTargetGateset uses controlled Z gates

It seems to be the controlled Z gates that are really used by Google on their machines, and these also give the best results in simulations, so in principle we should always use these.

In [ ]:
translated_bowtie = cirq.optimize_for_target_gateset(
    bowtie, context=cirq.TransformerContext(deep=True), gateset=cirq_google.GoogleCZTargetGateset()
)


### Choose qubits on the virtual device

Before execution you need to map the qubits used in defining the circuit onto the grid of the Sycamore processor. You can find the grid above or in the datasheet (https://quantumai.google/static/hardware/datasheet/weber.pdf).

You need to define your new qubits as a list of 'GridQubit' type qubits with coordinates as per the data sheet, and then use a dictionary to define the mapping.

In [ ]:
# 18 qubits, in a 3 by 6 block

device_qubits = [cirq.GridQubit(3,5),             # edges 1-2, first plaquette
                 cirq.GridQubit(3,6),
                 cirq.GridQubit(3,7),
                 cirq.GridQubit(2,5),
                 cirq.GridQubit(2,6),
                 cirq.GridQubit(2,7),

                 cirq.GridQubit(6,6),             # edges 3-4, second plaquette
                 cirq.GridQubit(6,4),             
                 cirq.GridQubit(6,5),             
                 cirq.GridQubit(5,6),
                 cirq.GridQubit(5,4),
                 cirq.GridQubit(5,5),
                 
                 cirq.GridQubit(5,7),             # charge measurement: encodes m, plus 2 to move ancilla around
                 cirq.GridQubit(6,7),
                 cirq.GridQubit(5,3),             # charge measurement: encodes R2, plus 2 to move ancilla around
                 cirq.GridQubit(6,3),
                 
                 cirq.GridQubit(4,5),             # BP for blue ribbon.
                 cirq.GridQubit(4,4),
                 cirq.GridQubit(2,4),             # BP for ribbon 2
                 cirq.GridQubit(3,4),
                 
                 cirq.GridQubit(4,6)]             # extra ancilla for moving ribbons around

qubit_map = {qubits[i]: device_qubits[i] for i in range(21)}


### Map the transformed circuit to the qubits you chose on the device

The `transform_qubits` function then uses the map to rewrite the circuit with the correct set of qubits.

In [ ]:
device_ready_bowtie = translated_bowtie.transform_qubits(lambda q: qubit_map[q])


print(device_ready_bowtie)


                                     ┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 ┌──┐                                                                                                                                                                                                                                                        

## **Execute** Your Circuit on the Quantum Virtual Machine

You can run the now device-ready circuit, as you would with any other `cirq.Engine` instance, by getting a sampler from it and using the `run` function on the circuits. Your choice of `repetitions` is intrinsically related to the accuracy of your simulated results. We recommend 3000 repetitions for trial runs, and 10,000 repetitions for accuracy-critical runs, but you can stick to one to ten repetitions when testing a code pipeline. You can read more about this in [this paper](https://arxiv.org/abs/2111.02396){:.external}.

In [ ]:
circuit = device_ready_bowtie

start = time.time()
results = sim_engine.get_sampler(processor_id).run(circuit, repetitions=repetitions)
elapsed = time.time() - start

print('Circuit successfully executed on your quantum virtual machine', processor_id)
print(f'QVM runtime: {elapsed:.04g}s ({repetitions} repetitions)')

Circuit successfully executed on your quantum virtual machine weber
QVM runtime: 6798s (10000 repetitions)



## **Process** Output

Use the results in array form to retrieve the group elements encoded in the edges and multiply out to find the vertex flux.

In [ ]:
# Results in array form: if there is a ribbon, pick out only the correct ones.

measurements = results.measurements['out']

if ribbon:
  measurements_filtered = []
  for mmt in measurements:
    if np.array_equal(mmt[-4:],[0,0,0,0]):
      measurements_filtered.append(mmt[:-4])
  measurements = measurements_filtered

repetitions_filtered = len(measurements)
print(repetitions_filtered)


914


In [ ]:
if measurement_type == 'flux':
  centre = 0

  for mmt in measurements:
    edge1 = np.kron(np.eye(2)[:,mmt[0]],
                  np.kron(np.eye(2)[:,mmt[1]],
                          np.eye(2)[:,mmt[2]]
                          ))
    edge2 = np.kron(np.eye(2)[:,mmt[3]],
                  np.kron(np.eye(2)[:,mmt[4]],
                          np.eye(2)[:,mmt[5]]
                          ))
    edge3 = np.kron(np.eye(2)[:,mmt[6]],
                  np.kron(np.eye(2)[:,mmt[7]],
                          np.eye(2)[:,mmt[8]]
                          ))
    edge4 = np.kron(np.eye(2)[:,mmt[9]],
                  np.kron(np.eye(2)[:,mmt[10]],
                          np.eye(2)[:,mmt[11]]
                          ))

    centre_flux = left_mult(edge2) @ left_mult(inv @ edge1) @ left_mult(inv @ edge3) @ edge4

    if np.array_equal(centre_flux,vector('e')):
      centre += 1

  print(centre/repetitions_filtered)




  

In [ ]:
if measurement_type == 'charge':
  count00 = 0
  count01 = 0
  count10 = 0
  count11 = 0

  for mmt in measurements:
    if np.array_equal(mmt[0:2],[0,0]):
      count00 += 1
    if np.array_equal(mmt[0:2],[0,1]):
      count01 += 1
    if np.array_equal(mmt[0:2],[1,0]):
      count10 += 1
    if np.array_equal(mmt[0:2],[1,1]):
      count11 += 1
  
  print(count00)
  print(count01)
  print(count10)
  print(count11)
  






In [ ]:
if measurement_type == 'both':
  output = np.zeros((4,8))
  
  for mmt in measurements:
    if np.array_equal(mmt[-2:],[0,0]):
      index1 = 0
    if np.array_equal(mmt[-2:],[0,1]):
      index1 = 1
    if np.array_equal(mmt[-2:],[1,0]):
      index1 = 2
    if np.array_equal(mmt[-2:],[1,1]):
      index1 = 3

    edge3 = np.kron(np.eye(2)[:,mmt[0]],
                  np.kron(np.eye(2)[:,mmt[1]],
                          np.eye(2)[:,mmt[2]]
                          ))
    edge4 = np.kron(np.eye(2)[:,mmt[3]],
                  np.kron(np.eye(2)[:,mmt[4]],
                          np.eye(2)[:,mmt[5]]
                          ))
    flux = left_mult(edge3) @ inv @ edge4

    for i in range(8):
      g = np.zeros(8)
      g[i] = 1
      if np.array_equal(flux,g):
        index2 = i
    
    output[index1][index2] += 1
  
  print(output)
    
  




[[88. 38. 65. 30. 32. 19. 27. 11.]
 [39. 25. 30. 18. 16. 11. 16.  6.]
 [69. 37. 71. 40. 19. 17. 20. 14.]
 [38. 26. 39. 22. 13.  6.  7.  5.]]


As a check, this is an ideal simulation using the same device transformed circuit.

In [ ]:
if measurement_type == 'both' and ribbon:
    
  output = np.zeros((4,8))

  for repetitions in range(10):
    # run the circuit
    
    simulator = cirq.Simulator()
    ideal_measurement = simulator.run(device_ready_bowtie).measurements['out'][0]

    if np.array_equal(ideal_measurement[-4:],[0,0,0,0]):

      ideal_mmt = ideal_measurement[:-4]
      
      if np.array_equal(ideal_mmt[-2:],[0,0]):
        index1 = 0
      if np.array_equal(ideal_mmt[-2:],[0,1]):
        index1 = 1
      if np.array_equal(ideal_mmt[-2:],[1,0]):
        index1 = 2
      if np.array_equal(ideal_mmt[-2:],[1,1]):
        index1 = 3

      edge3 = np.kron(np.eye(2)[:,ideal_mmt[0]],
                    np.kron(np.eye(2)[:,ideal_mmt[1]],
                            np.eye(2)[:,ideal_mmt[2]]
                            ))
      edge4 = np.kron(np.eye(2)[:,ideal_mmt[3]],
                    np.kron(np.eye(2)[:,ideal_mmt[4]],
                            np.eye(2)[:,ideal_mmt[5]]
                            ))
      flux = left_mult(edge3) @ inv @ edge4

      for i in range(8):
        g = np.zeros(8)
        g[i] = 1
        if np.array_equal(flux,g):
          index2 = i
    
      output[index1][index2] += 1
    
    
      
  print(output)

[[1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
